In [ ]:
# from astropy.coordinates import SkyCoord

# NAMES = ["J033227.0-274105", "052.696667-27.988028"]
# coords = SkyCoord([SkyCoord.from_name(name) for name in NAMES])
# coords
# RA, DEC = coords.ra.deg, coords.dec.deg
RADEC = [(53.11255, -27.68482), (52.69695, -27.98828)]

In [ ]:
import lsdb

rows = []
for ra, dec in RADEC:
    cat = lsdb.open_catalog(
        "/sdf/data/rubin/user/kostya/hats/dia_object_lc_15k",
        search_filter=lsdb.ConeSearch(ra, dec, radius_arcsec=0.5)
    )
    df = cat.compute()
    assert len(df) == 1, f"{len(df) = }, {ra = }, {dec = }"
    row = df.iloc[0]
    rows.append(row)
rows

In [ ]:
lcs = [row["diaObjectForcedSource"].query(
    "~psfFlux_flag"
    " and ~psfDiffFlux_flag"
    " and ~pixelFlags_suspect"
    " and ~pixelFlags_saturated"
    " and ~pixelFlags_cr"
    " and ~pixelFlags_bad"
) for row in rows]
display(lcs[0])
display(lcs[1])

In [ ]:
import matplotlib.pyplot as plt

COLORS = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

for row, lc in zip(rows, lcs):
    plt.figure()
    for band in "ugrizy":
        data = lc.query("band == @band and psfMagErr < 0.2")
        plt.errorbar(data["midpointMjdTai"], data["psfMag"], yerr=data["psfMagErr"], label=f"{band}", fmt='o', color=COLORS[band])

    plt.gca().invert_yaxis()
    plt.xlabel(f"MJD")
    plt.ylabel("PSF mag")
    plt.title(f"diaObjectID {row.diaObjectId}\nRA=${row.ra:.5f}$, Dec=${row.dec:.5f}$")
    plt.legend(loc='lower right')
    plt.savefig(f"{row.diaObjectId}.pdf")